In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
df = pd.read_csv('data_One_hot.csv')
df = df.drop(columns=['UserID'])

X = df.drop('CourseCompletion', axis=1)
y = df['CourseCompletion']

numerical_features = ['TimeSpentOnCourse', 'NumberOfVideosWatched', 'NumberOfQuizzesTaken', 'QuizScores', 'CompletionRate','DeviceType','is_health','is_programming','is_arts','is_science']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features)
    ],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=50)

In [4]:
df

,TimeSpentOnCourse,NumberOfVideosWatched,NumberOfQuizzesTaken,QuizScores,CompletionRate,DeviceType,CourseCompletion,is_health,is_arts,is_science,is_programming
0,29.979719,17,3,50.365656,20.860773,1,0,1,0,0,0
1,27.802640,1,5,62.615970,65.632415,1,0,0,1,0,0
2,86.820485,14,2,78.458962,63.812007,1,1,0,1,0,0
3,35.038427,17,10,59.198853,95.433162,0,1,0,0,1,0
4,92.490647,16,0,98.428285,18.102478,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
8995,37.445225,14,4,54.469359,32.990704,1,0,1,0,0,0
8996,48.631443,7,7,59.413257,0.254625,0,0,0,0,1,0
8997,38.212512,3,3,69.508297,70.188159,1,0,1,0,0,0
8998,70.048665,13,10,79.655182,72.975225,1,1,1,0,0,0


In [5]:
model = Sequential([
    Dense(80, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(40, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

model.fit(X_train, y_train, epochs=200, batch_size=36, validation_split = 0.1, callbacks=[early_stopping])

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))
print(f'AUC-ROC: {roc_auc_score(y_test, y_pred_prob)}')



Epoch 1/200


180/180 [==============================] - 5s 8ms/step - loss: 0.5069 - accuracy: 0.7503 - val_loss: 0.4249 - val_accuracy: 0.8125
Epoch 2/200
180/180 [==============================] - 1s 5ms/step - loss: 0.4292 - accuracy: 0.8079 - val_loss: 0.4027 - val_accuracy: 0.8222
Epoch 3/200
180/180 [==============================] - 1s 5ms/step - loss: 0.4126 - accuracy: 0.8179 - val_loss: 0.3869 - val_accuracy: 0.8278
Epoch 4/200
180/180 [==============================] - 1s 5ms/step - loss: 0.4012 - accuracy: 0.8256 - val_loss: 0.3773 - val_accuracy: 0.8347
Epoch 5/200
180/180 [==============================] - 1s 5ms/step - loss: 0.3923 - accuracy: 0.8347 - val_loss: 0.3724 - val_accuracy: 0.8403
Epoch 6/200
180/180 [==============================] - 1s 4ms/step - loss: 0.3811 - accuracy: 0.8373 - val_loss: 0.3659 - val_accuracy: 0.8417
Epoch 7/200
180/180 [==============================] - 1s 5ms/step - loss: 0.3748 - accuracy: 0.8389 - val_loss: 0.3562 - val_accuracy: 0.